# 01 — Data Cleaning & Preparation

> **Objective:** To load the raw public transit delay dataset, assess data quality, perform cleaning and feature engineering, and save a processed dataset for downstream exploratory analysis and modeling.

This notebook outlines the following stages:
1. [**Dataset overview**](#dataset-overview) — loading raw data and inspecting structure  
2. [**Missing values analysis**](#missing-values-analysis) — assessing completeness and handling nulls  
3. [**Data cleaning steps**](#data-cleaning-steps) — addressing inconsistencies, types, and outliers  
4. [**Feature engineering**](#feature-engineering) — creating derived features for analysis  
5. [**Save cleaned dataset**](#save-cleaned-dataset) — exporting to `data/processed/`  

> **Note:** Section links work in Jupyter or nbviewer; they may not render in static GitHub previews.

---
### 🧠 Project Context

This notebook is the first step in the **Public Transit Delay EDA** project. Clean, well-structured data is essential for reliable exploratory analysis and any subsequent modeling. All transformations applied here are documented so that the pipeline is reproducible.

---
### 🧰 Imports <a id="imports"></a>

Core libraries for data loading, manipulation, and cleaning:

- **pandas** — data loading, tabular manipulation, and export  
- **numpy** — numerical operations where needed  
- **pathlib / os** — path handling for reading and writing files  

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

---
### 📥 Dataset Overview <a id="dataset-overview"></a>

Load the raw dataset from `data/raw/` and inspect its structure: shape, column names, dtypes, and a sample of rows.  

This confirms that the import completed successfully and provides a first look at the variables available for analysis.

In [2]:
raw_path = Path("../data/raw/public_transport_delays.csv")
df = pd.read_csv(raw_path)
print("Shape:", df.shape)
df.head()

Shape: (2000, 24)


,trip_id,date,time,transport_type,route_id,origin_station,destination_station,scheduled_departure,scheduled_arrival,actual_departure_delay_min,...,wind_speed_kmh,precipitation_mm,event_type,event_attendance_est,traffic_congestion_index,holiday,peak_hour,weekday,season,delayed
0,T00000,2023-01-01,05:00:00,Tram,Route_15,Station_31,Station_6,05:02:00,05:55:00,12,...,46,13.0,NaN,500,81,0,1,6,Winter,0
1,T00001,2023-01-01,05:15:00,Metro,Route_12,Station_49,Station_32,05:16:00,05:55:00,15,...,11,11.4,NaN,0,53,0,0,6,Autumn,1
2,T00002,2023-01-01,05:30:00,Bus,Route_16,Station_29,Station_42,05:33:00,06:17:00,0,...,31,14.1,Sports,0,67,1,0,6,Autumn,0
3,T00003,2023-01-01,05:45:00,Tram,Route_19,Station_26,Station_18,05:49:00,06:08:00,15,...,41,6.4,NaN,500,84,0,0,6,Winter,1
4,T00004,2023-01-01,06:00:00,Tram,Route_8,Station_18,Station_15,06:00:00,06:35:00,-1,...,30,18.5,NaN,500,46,0,0,6,Spring,1


| Column | Description |
|--------|-------------|
| `trip_id` | Unique trip identifier |
| `date` | Trip date |
| `time` | Trip start time |
| `transport_type` | Bus, Tram, Metro, or Train |
| `route_id` | Route identifier (e.g. Route_1, Route_2) |
| `origin_station`, `destination_station` | Start and end station IDs |
| `scheduled_departure`, `scheduled_arrival` | Planned departure/arrival times |
| `actual_departure_delay_min`, `actual_arrival_delay_min` | Delay in minutes (negative = early) |
| `weather_condition` | Clear, Rain, Snow, Storm, Fog, Cloudy |
| `temperature_C`, `humidity_percent`, `wind_speed_kmh`, `precipitation_mm` | Weather variables |
| `event_type` | None, Sports, Concert, Parade, Protest, Festival |
| `event_attendance_est` | Estimated event attendance |
| `traffic_congestion_index` | Congestion level (0–100) |
| `holiday` | 1 if holiday, 0 otherwise |
| `peak_hour` | 1 if peak, 0 otherwise |
| `weekday` | Day of week (0–6) in raw data |
| `season` | Winter, Spring, Summer, Autumn |
| `delayed` | 1 if trip was delayed (arrival delay > 0), 0 otherwise |

---
### 🧾 Missing Values Analysis <a id="missing-values-analysis"></a>

Summarize the dataset structure with `df.info()` and count nulls per column.  
Identifying missing values is essential before cleaning so that imputation or removal strategies can be applied consistently.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   trip_id                     2000 non-null   object 
 1   date                        2000 non-null   object 
 2   time                        2000 non-null   object 
 3   transport_type              2000 non-null   object 
 4   route_id                    2000 non-null   object 
 5   origin_station              2000 non-null   object 
 6   destination_station         2000 non-null   object 
 7   scheduled_departure         2000 non-null   object 
 8   scheduled_arrival           2000 non-null   object 
 9   actual_departure_delay_min  2000 non-null   int64  
 10  actual_arrival_delay_min    2000 non-null   int64  
 11  weather_condition           2000 non-null   object 
 12  temperature_C               2000 non-null   float64
 13  humidity_percent            2000 

In [4]:
df.isnull().sum()

trip_id                          0
date                             0
time                             0
transport_type                   0
route_id                         0
origin_station                   0
destination_station              0
scheduled_departure              0
scheduled_arrival                0
actual_departure_delay_min       0
actual_arrival_delay_min         0
weather_condition                0
temperature_C                    0
humidity_percent                 0
wind_speed_kmh                   0
precipitation_mm                 0
event_type                    1173
event_attendance_est             0
traffic_congestion_index         0
holiday                          0
peak_hour                        0
weekday                          0
season                           0
delayed                          0
dtype: int64

#### 🔎 *Summary*

**Only `event_type` has missing values** (1,173 of 2,000 rows). No event was recorded for those trips. We will **fill these with the string `"None"`** so that EDA and modeling can treat "no event" as a distinct category. All other columns are complete.

---
### 🧹 Data Cleaning Steps <a id="data-cleaning-steps"></a>

Apply cleaning steps such as:
- Correcting data types (dates, categories, numeric)  
- Handling or imputing missing values  
- Removing or flagging duplicates  
- Addressing obvious outliers or invalid values  

In [5]:
# Parse date and build datetime for time-based features
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"], errors="coerce")

# Fill missing event_type with "None" (no event)
df["event_type"] = df["event_type"].fillna("None")

# Drop duplicate rows if any (by trip_id)
n_before = len(df)
df = df.drop_duplicates(subset=["trip_id"], keep="first")
print(f"Dropped {n_before - len(df)} duplicate trip(s). Rows: {len(df)}")

# Ensure numeric delay columns are int (already are)
df[["actual_departure_delay_min", "actual_arrival_delay_min"]] = df[
    ["actual_departure_delay_min", "actual_arrival_delay_min"]
].astype(int)
df.head(3)

Dropped 0 duplicate trip(s). Rows: 2000


,trip_id,date,time,transport_type,route_id,origin_station,destination_station,scheduled_departure,scheduled_arrival,actual_departure_delay_min,...,precipitation_mm,event_type,event_attendance_est,traffic_congestion_index,holiday,peak_hour,weekday,season,delayed,datetime
0,T00000,2023-01-01,05:00:00,Tram,Route_15,Station_31,Station_6,05:02:00,05:55:00,12,...,13.0,None,500,81,0,1,6,Winter,0,2023-01-01 05:00:00
1,T00001,2023-01-01,05:15:00,Metro,Route_12,Station_49,Station_32,05:16:00,05:55:00,15,...,11.4,None,0,53,0,0,6,Autumn,1,2023-01-01 05:15:00
2,T00002,2023-01-01,05:30:00,Bus,Route_16,Station_29,Station_42,05:33:00,06:17:00,0,...,14.1,Sports,0,67,1,0,6,Autumn,0,2023-01-01 05:30:00


---
### ⚙️ Feature Engineering <a id="feature-engineering"></a>

Create derived features that may be useful for EDA and modeling, for example:
- Time-based: hour of day, day of week, month, peak vs off-peak  
- Delay-related: delay bins, on-time vs delayed flag  
- Route or line aggregates  

In [6]:
# Hour of day (0–23) and day of week (0=Monday, 6=Sunday)
df["hour"] = df["datetime"].dt.hour
df["day_of_week"] = df["datetime"].dt.dayofweek

# Primary delay for analysis: use arrival delay (passenger-facing)
df["delay_minutes"] = df["actual_arrival_delay_min"].copy()

# Delay category for interpretation
def delay_category(minutes):
    if minutes <= 0:
        return "On time"
    if minutes <= 5:
        return "Slight (1–5 min)"
    if minutes <= 15:
        return "Moderate (6–15 min)"
    return "Severe (15+ min)"

df["delay_category"] = df["delay_minutes"].apply(delay_category)

# Preview engineered columns
df[["datetime", "hour", "day_of_week", "delay_minutes", "delay_category"]].head(5)

,datetime,hour,day_of_week,delay_minutes,delay_category
0,2023-01-01 05:00:00,5,6,3,Slight (1–5 min)
1,2023-01-01 05:15:00,5,6,9,Moderate (6–15 min)
2,2023-01-01 05:30:00,5,6,0,On time
3,2023-01-01 05:45:00,5,6,10,Moderate (6–15 min)
4,2023-01-01 06:00:00,6,6,14,Moderate (6–15 min)


---
### 💾 Save Cleaned Dataset <a id="save-cleaned-dataset"></a>

Export the cleaned and engineered dataset to `data/processed/` so that downstream notebooks (e.g. EDA) can load it without re-running cleaning steps.

In [7]:
out_path = Path("../data/processed/transit_delays_cleaned.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_path, index=False)
print(f"Saved {len(df)} rows to {out_path}")

Saved 2000 rows to ../data/processed/transit_delays_cleaned.csv
